In [ ]:
# 1. 마운트 설정
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 2. 업로드 한 파일이 있는 경로를 현재 작업디렉터리로 연결함
import os
os.chdir("drive/My Drive/tb_app/")

In [ ]:
# 3. 작업디렉터리가 가진 목록을 화면에 표시
!ls

In [ ]:
# 선수작업2-1

import numpy as np
import scipy as sp
import pandas as pd

In [ ]:
# 선수작업2-2

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

mpl.rcParams['axes.unicode_minus'] = False  # -기호 깨짐방지

In [ ]:
# 선수작업2-3

# 나눔폰트 유무 확인
sys_font=fm.findSystemFonts()
print(f"sys_font number: {len(sys_font)}")
print(sys_font)

nanum_font = [f for f in sys_font if 'Nanum' in f]
print(f"nanum_font number: {len(nanum_font)}")

In [ ]:
# 선수작업2-4

nanum_font

In [ ]:
# 선수작업2-5
# 나눔폰트 없는 경우 설치. 있는 경우 skip
!apt-get update -qq
!apt-get install fonts-nanum* -qq

In [ ]:
# 선수작업2-5-1
# fm._rebuild() 에러발생시 실행후 런타임 다시 시작
# Cleaning the matplotlib cache directory
!rm ~/.cache/matplotlib -rf

In [ ]:
# 선수작업2-6
# 한글폰트 지정
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=10)
# fm._rebuild()

# 그래프에 retina display 적용

%config InlineBackend.figure_format = 'retina'

# Colab 의 한글 폰트 설정

plt.rc('font', family='NanumBarunGothic')

In [ ]:
from sklearn.datasets import load_iris
ds_iris = load_iris()
df_iris = pd.DataFrame(ds_iris['data'], columns=ds_iris['feature_names'])
df_iris['iris_name'] = [ds_iris['target_names'][x] for x in ds_iris['target']]
df_iris = df_iris.rename({'sepal length (cm)': 'sepal_length',
                          'sepal width (cm)': 'sepal_width',
                          'petal length (cm)': 'petal_length',
                          'petal width (cm)': 'petal_width'}, axis='columns')
print(df_iris.head())

In [ ]:
# 학습데이터, 시험데이터, 검증데이터로 나누기위해서 데이터를 섞음
df_is = df_iris.sample(frac=1)  # frac=1 : 모든 데이터를 랜덤하게 섞음
print(df_is.head())

In [ ]:
# 학습데이터, 시험데이터, 검증데이터 준비
# 학습데이터, 시험데이터, 검증데이터 비율 - 60:20:20
df_it = df_is.iloc[0:120, :]  # 학습데이터-90, 시험데이터-30
print(df_it.head())

In [ ]:
df_iv = df_is.iloc[120:, :]  # 검증데이터-30
print(df_iv.head())

In [ ]:
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split

# 학습 데이터와 시험 데이터로 나눔 - 비율은 생략시 75% : 25%
# test_size=.25 는 학습데이터 0.75 - 90개, 시험데이터 0.25 - 30개
# train_data, test_data, train_label, test_label = \
# train_test_split(df_it[df_it.columns[0:4]], df_it[df_it.columns[4]], test_size=.25)
train_data, test_data, train_label, test_label = \
 train_test_split(df_it[df_it.columns[0:4]], df_it[df_it.columns[4]])

# 난수 시드 고정
np.random.seed(202005)

# 데이터 학습
clf = svm.SVC()  # 분류기 - 서포트벡터머신 사용
clf.fit(train_data, train_label)

# 데이터 예측
pre = clf.predict(test_data)

# 정답률 구함
ac_score = metrics.accuracy_score(test_label, pre)
print("정답률 =", ac_score)

In [ ]:
# 학습결과를 파일로 저장
import joblib
joblib.dump(clf, "iris_model.joblib")

In [ ]:
# 저장된 학습결과를 불러와서 검증데이터에 적용: 학습 검증
iris_pre = joblib.load("iris_model.joblib")
result = iris_pre.predict(df_iv[df_iv.columns[0:4]])
true_data = df_iv[df_iv.columns[4]].values

# 판별한 결과 중 일부 출력해 봄
for i in range(20):
    print("predict : " + result[i] + " - " + "True : " + true_data[i])

# 판별한 결과로 정답률 계산
rlist = 0
for i in range(result.size):
    if(result[i] == true_data[i]):
        rlist += 1

tr = rlist / result.size

print("Ture rate : " + str(tr))


In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

np.random.seed(202005)  # 재현성확보에 필요

# 분류기 이름 -> (선 스타일, 분류기 인스턴스)
map_classifier = {
        '로지스틱 회귀': LogisticRegression(),
        '나이브 베이즈': GaussianNB(),
        '디시전 트리': DecisionTreeClassifier(max_depth=5),
        '랜덤 포레스트': RandomForestClassifier(max_depth=5, n_estimators=10,
                                          max_features=1),
        '서포트벡터머신': SVC(probability=True)
}

# 데이터프레임 행을 학습/시험셋으로 나눈다.
data, label = df_iris[df_iris.columns[:4]], df_iris[df_iris.columns[4]]
train_data, test_data, train_label, test_label = train_test_split(data, label)

print('분류기 성능 비교')
for c_name, model in map_classifier.items():
    model.fit(train_data, train_label)
    # 열마다 True/False 예측값을 저장한다.
    preds = model.predict_proba(test_data)	 # 모양: (120, 2)
    pre = model.predict(test_data)
    pred = pd.Series(preds[:, 1])
    ac_score = metrics.accuracy_score(test_label, pre)
    print("분류기: {0:5s}, 정답률 = {1:.5f}".format(c_name, ac_score))

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

np.random.seed(202005)  # 재현성확보에 필요

# 분류기 이름 : (선 스타일, 분류기 인스턴스)
map_classifier = {
        '로지스틱 회귀': LogisticRegression(),
        '나이브 베이즈': GaussianNB(),
        '디시전 트리': DecisionTreeClassifier(max_depth=5),
        '랜덤 포레스트': RandomForestClassifier(max_depth=5, n_estimators=10,
                                          max_features=1),
        '서포트벡터머신': SVC(probability=True)
}

# 데이터프레임 행을 학습/시험셋으로 나눈다. 데이터는 자동으로 랜덤하게 섞어서 나눔
data, label = df_iris[df_iris.columns[:4]], df_iris[df_iris.columns[4]]
train_data, test_data, train_label, test_label = train_test_split(data, label)

print('분류기 성능 비교')
for c_name, model in map_classifier.items():
    model.fit(train_data, train_label)
    # 열마다 True/False 예측값을 저장한다.
    preds = model.predict_proba(test_data)	 # 모양: (120, 2)
    pre = model.predict(test_data)
    pred = pd.Series(preds[:, 1])
    ac_score = metrics.accuracy_score(test_label, pre)
    print("분류기: {0:5s}, 정답률 = {1:.5f}".format(c_name, ac_score))

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

np.random.seed(202005)  # 재현성확보에 필요

# 분류기 이름 : (선 스타일, 분류기 인스턴스)
map_classifier = {
        '로지스틱 회귀': ('-', LogisticRegression()),
        '나이브 베이즈': ('--', GaussianNB()),
        '디시전 트리': ('-.', DecisionTreeClassifier()),
        '랜덤 포레스트': (':', RandomForestClassifier()),
        '서포트벡터머신': ('o', SVC(probability=True))
}

# 데이터프레임 행을 학습/시험셋으로 나눈다.
data, label = df_iris[df_iris.columns[:3]], (df_iris['iris_name'] == 'virginica')
train_data, test_data, train_label, test_label = \
    train_test_split(data, label, test_size=.8)

for c_name, (line_s, model) in map_classifier.items():
    model.fit(train_data, train_label)
    # 열마다 True/False 예측값을 저장한다.
    preds = model.predict_proba(test_data)	 # 모양: (120, 2)
    pred = pd.Series(preds[:, 1])
    fpr, tpr, thresholds = roc_curve(test_label, pred)
    auc_score = auc(fpr, tpr)
    label = '%s: auc=%f' % (c_name, auc_score)
    plt.plot(fpr, tpr, line_s, linewidth=5, label=label)
    print(label)

plt.legend(loc="lower right")
plt.title('분류기 성능 비교')

plt.plot([0, 1], [0, 1], 'k--')  # x=y line.  Visual aid
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('거짓 양성비')
plt.ylabel('참 양성비')
plt.show()

In [ ]:
# 버섯 종류 구분
# 1. mushroom 데이터 다운로드
import urllib.request as req

data = "data/mushroom.csv"
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data"
req.urlretrieve(url, data)
print("done")

In [ ]:
# 2. 전처리, 문자데이터를 숫자로 변환하기
# (1) 데이터로드
import pandas as pd
df_mr = pd.read_csv("data/mushroom.csv", header=None)

label_mr = []
data_mr = []

# (2) 분류기 학습을 위해 기호를 숫자로 변환
for row_index, row in df_mr.iterrows():
    label_mr.append(row.loc[0])
    row_data = []
    for v in row.loc[1:]:
        row_data.append(ord(v))
    data_mr.append(row_data)

In [ ]:
data_mr

In [ ]:
# 3. 버섯종류 판별 모델링
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm, metrics

np.random.seed(1909)  # 재현성확보에 필요
# (1) 모델 비교에 사용할 분류기 딕셔너리로 저장
# 분류기 이름 : (선 스타일, 분류기 인스턴스)
map_classifier = {
        '로지스틱 회귀': LogisticRegression(max_iter=5000),
        '나이브 베이즈': GaussianNB(),
        '결정 트리': DecisionTreeClassifier(max_depth=5),
        '랜덤 포레스트': RandomForestClassifier(max_depth=5),
        'SVM': svm.SVC()
}

# (2) 학습 전용과 테스트 전용 데이터로 나누기
train_data, test_data, train_label, test_label = \
    train_test_split(data_mr, label_mr)

# 분류기 학습 및 비교
for name, model in map_classifier.items():
    # (3) 데이터 학습시키기
    model.fit(train_data, train_label)
    # (4) 데이터 예측하기
    predict = model.predict(test_data)
    # (5) 결과 테스트하기
    ac_score = metrics.accuracy_score(test_label, predict)
    cl_report = metrics.classification_report(test_label, predict)
    print("분류기 :", name)
    print("정답률 =", ac_score)
    print("리포트 =\n", cl_report)

In [ ]:
# 4. 버섯종류 판별 학습
import pandas as pd
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split

np.random.seed(1909)  # 재현성확보에 필요

# (3) 학습 전용과 테스트 전용 데이터로 나누기
train_data, test_data, train_label, test_label = \
    train_test_split(data_mr, label_mr)
# 데이터 학습
model = svm.SVC()
model.fit(train_data, train_label)
# 데이터 예측
predict = model.predict(test_data)
# 결과 테스트하기 --- (※6)
ac_score = metrics.accuracy_score(test_label, predict)
cl_report = metrics.classification_report(test_label, predict)
print("정답률 =", ac_score)
print("리포트 =\n", cl_report)

In [ ]:
# 5. 버섯종류 판별 학습결과 저장
import joblib
joblib.dump(model, "mushroom.joblib")

In [ ]:
# 6. 파일로 저장된 학습결과로 판별하기
mushroom_pre = joblib.load("mushroom.joblib")
result = mushroom_pre.predict(train_data)
true_dat = train_label

rlist = 0
for i in range(result.size):
    if(result[i] == true_dat[i]):
        rlist += 1

tr = rlist / result.size

print("Ture rate : " + str(tr))

In [ ]:
# 회귀분석 - 당뇨병 진행 예측
# 1. 데이터 획득
import sklearn.datasets
from sklearn.preprocessing import normalize

diabetes = sklearn.datasets.load_diabetes()
X = normalize(diabetes['data'])
Y = diabetes['target']

In [ ]:
# mape를 구하는 mean_absolute_percentage_error() 함수 작성
import numpy as np


def mean_absolute_percentage_error(actual, pred):
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [ ]:
# 2. 모델링
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error

np.random.seed(190811)  # 재현성확보에 필요
# (1) 모델링에 사용할 모델
map_regression = {
        '선형 회귀': LinearRegression(),
        '라쏘 회귀': Lasso(),
        '릿지 회귀': Ridge(),
        '엘라스틱 넷 회귀': ElasticNet(),
        '서포트 벡터 회귀': SVR(C=1.0, epsilon=0.2),
        '랜덤 포레스트 회귀': RandomForestRegressor(max_depth=5)
}

# 학습 전용과 테스트 전용 데이터로 나누기
train_X, test_X, train_Y, test_Y = train_test_split(X, Y)

# 데이터 학습
for name, model in map_regression.items():
    # (3) 데이터 학습
    model.fit(train_X, train_Y)
    # (4) 데이터 예측
    predict = model.predict(test_X)
    # (5) 결과 테스트
    corr = round(pd.Series(predict).corr(pd.Series(test_Y)), 3)
    rsquared = round(r2_score(test_Y, predict), 3)
    print("\n")
    print(name)
    print("상관 계수:", corr, "결정 계수-R2:", rsquared)
    if(name not in ['서포트 벡터 회귀', '랜덤 포레스트 회귀']):
        print("선형 계수:")
        print(model.coef_)

    #머신러닝 회귀분석 예측모델 평가
    mape = mean_absolute_percentage_error(test_Y, predict)
    rmse = mean_squared_error(test_Y, predict)
    mae = mean_absolute_error(test_Y, predict)

    print("MAPE:", mape, "RMSE:", rmse, "MAE:", mae)


    # 시각화
    plt.scatter(predict, test_Y)
    plt.title("선형 회귀 결과: %s. 상관계수=%f, $R^2$ 점수=%f" % (name, corr, rsquared))
    plt.xlabel("예측값")
    plt.ylabel("실제값")
    # 비교를 위해 x=y 라인 추가
    plt.plot(test_Y , test_Y , 'k--')
    plt.show()